In [1]:
import expansion_models

import collections
import random
import math
import string

import rdkit
from rdkit import Chem
from rdkit.Chem.EState import Fingerprinter
from rdkit.Chem import Descriptors
from rdkit.Chem import rdFMCS
from rdkit.Chem.rdmolops import RDKFingerprint
from rdkit.Chem.Fingerprints import FingerprintMols
from rdkit import DataStructs
from rdkit.Avalon.pyAvalonTools import GetAvalonFP

import numpy as np
import pandas as pd
import pubchempy as pc
def randomSmiles(m1):
    m1.SetProp("_canonicalRankingNumbers", "True")
    idxs = list(range(0,m1.GetNumAtoms()))
    random.shuffle(idxs)
    for i,v in enumerate(idxs):
        m1.GetAtomWithIdx(i).SetProp("_canonicalRankingNumber", str(v))
    return Chem.MolToSmiles(m1)

m1 = Chem.MolFromSmiles("CNOPc1ccccc1")
m2 = Chem.MolFromSmiles("O=C(O)C(=O)CCC(=O)O")
m3 = Chem.MolFromSmiles('C(C(C(C(=O)[O-])O)C(=O)[O-])C(=O)[O-]')

s = np.empty([362], dtype=('U256'))
for i in range(0, len(s)):
    smiles = randomSmiles(m1)
    s[i] = smiles
e= np.empty([362], dtype=('U256'))
for i in range(0, len(e)):
    e[i] = ''.join(random.choices(string.ascii_uppercase + string.digits, k=8))

def fingerprint_products(input_df):  # fingerprints all products in a given df
    """
    Args:
        input_df : pandas dataframe with smiles in column named 'SMILES'

    Returns:
        input_df : with added mol and fingerprint columns
    """

    mol_list = []
    fp_list = []

    for index, row in input_df.iterrows():
        # get mols from SMILES and add mols to list
        mol_list.append(Chem.rdmolfiles.MolFromSmiles(row['SMILES']))
        fp_list.append(FingerprintMols.FingerprintMol(Chem.rdmolfiles.MolFromSmiles(
            row['SMILES'])))  # get fingerprints from mols and and fingerprints to list

    input_df.insert(1, column='Mol', value=mol_list)
    input_df.insert(2, column='Fingerprint', value=fp_list)

    return input_df
    

In [2]:
master_df = pd.read_csv('../datasets/master_dataframe_metacyc.csv', compression='gzip')
master_df.drop(columns='Fingerprint', inplace=True)
master_df= fingerprint_products(master_df)
prom_df = master_df[master_df['reacts'] ==1] 
prom_df.head()

,Enzyme,Mol,Fingerprint,product,reacts,PubChemID,SMILES,n_C,n_H,n_O,...,MW,Dist,Promiscuous,enzyme_class_1,enzyme_class_2,enzyme_class_3,enzyme_class_4,enzyme_class_5,enzyme_class_6,enzyme_class_7
0,1.14.14.80,<rdkit.Chem.rdchem.Mol object at 0x000001FF0BA...,"[1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, ...",CPD-10515,1.0,25201835,CCCCCCCCC(C(CCCCCCCC(=O)[O-])O)O,18,35,4,...,315.474,0.714718,1.0,1,0,0,0,0,0,0
1,1.14.14.80,<rdkit.Chem.rdchem.Mol object at 0x000001FF0EB...,"[0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, ...",PALMITATE,1.0,504166,CCCCCCCCCCCCCCCC(=O)[O-],16,31,2,...,255.422,0.714718,1.0,1,0,0,0,0,0,0
2,1.14.14.80,<rdkit.Chem.rdchem.Mol object at 0x000001FF0EB...,"[0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, ...",OLEATE-CPD,1.0,5460221,CCCCCCCCC=CCCCCCCCC(=O)[O-],18,33,2,...,281.460,0.714718,1.0,1,0,0,0,0,0,0
3,1.14.14.80,<rdkit.Chem.rdchem.Mol object at 0x000001FF0EB...,"[0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, ...",STEARIC_ACID,1.0,3033836,CCCCCCCCCCCCCCCCCC(=O)[O-],18,35,2,...,283.476,0.714718,1.0,1,0,0,0,0,0,0
4,1.14.14.80,<rdkit.Chem.rdchem.Mol object at 0x000001FF0EB...,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...",CPD-10514,1.0,19746553,CCCCCCCCC1C(O1)CCCCCCCC(=O)[O-],18,33,3,...,297.459,0.714718,1.0,1,0,0,0,0,0,0


In [4]:
def dist_for_expansion(prom_df, smiles, n=20):
    """
    Args:
        prom_df : pandas dataframe with all promiscuous enzymatic reactions
        pubchem_cid : cid of query molecule
        num_similar (int) : number of similar compounds to retrieve for tree search.
                            If input is 'None', default is 20 molecules.
                            A higher number will make search time longer.

    Returns:
        selected (df) : df with selected promiscuous enzyme pairs to use in tree search
    """
    

    if len(smiles) == 0:
        raise 'query compound SMILES string could not be retrieved'
    else:
        pass
    
    # get molecule and fingerprint for query compound
    # using rdkit
    mol = Chem.rdmolfiles.MolFromSmiles(smiles)
    fingerprint = FingerprintMols.FingerprintMol(mol)

    prom_df['ExpanDist'] = ''

    for index, row in prom_df.iterrows():
        # get similarity between query compound and each molecule 
        # in df
        comp = DataStructs.FingerprintSimilarity(
            row['Fingerprint'], fingerprint, metric=DataStructs.TanimotoSimilarity)
        prom_df['ExpanDist'].loc[index] = comp

    prom_df.sort_values(by='ExpanDist', ascending=False, inplace=True)

    selected = prom_df.iloc[:n, :].copy()
    

    return selected

def check_for_known(all_enz_df, mol_fingerprint, threshold):
    """
    Args:
        all_enz_df : pandas dataframe with all known enzymatic reactions
        mol_fingerprint : RDKit fingerprint of query molecule
        threshold (float, 0-1.0) : cut-off value for similarity.
                                    Recommend 0.95 or above for truly similar compounds.

    Returns:
        known_df : df with known enzymatic transformations of this molecule
                    ordered from most similar molecule to least similar
        or
        printed message : indication to begin searching through promiscuous enzymes

    """
    # drop any columns leftover from the last iteration
    bad_cols = ['Known', 'Mol', 'Fingerprint']

    for col in bad_cols:
        if col in all_enz_df.columns:
            all_enz_df.drop(columns=col, inplace=True)
        else:
            continue

    fingerprint = mol_fingerprint

    # fingerprint the input dataframe and return it
    input_df = fingerprint_products(all_enz_df)

    input_df['Known'] = ''

    for index, row in input_df.iterrows():
        # get similarity between query compound and each molecule 
        # in df
        similarity = DataStructs.FingerprintSimilarity(
            fingerprint, row['Fingerprint'], metric=DataStructs.TanimotoSimilarity)
        input_df['Known'].loc[index] = similarity

    # select only reactions that have product similarity to query
    # greater than the threshold
    known_df = input_df[input_df['Known'] >= threshold]

    if len(known_df) > 0:
        known_df.sort_values(by='Known', ascending=False, inplace=True)
        result = known_df
    else:
        # call to promiscuous search code here
        result = print('No known enzymes. Begin promiscuous search.')

    return result

In [7]:
dsf = 'CC(=O)NC(CCC(=O)OP(=O)(O)O)C(=O)O'

test_df = dist_for_expansion(prom_df, dsf, 3)
test_df.head()

C:\Users\phili\Miniconda3\lib\site-packages\ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\phili\Miniconda3\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
C:\Users\phili\Miniconda3\lib\site-packages\ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\phili\Miniconda3\lib\site-packages

,Enzyme,Mol,Fingerprint,product,reacts,PubChemID,SMILES,n_C,n_H,n_O,...,Dist,Promiscuous,enzyme_class_1,enzyme_class_2,enzyme_class_3,enzyme_class_4,enzyme_class_5,enzyme_class_6,enzyme_class_7,ExpanDist
951,3.4.13.18,<rdkit.Chem.rdchem.Mol object at 0x000001FF0EC...,"[0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, ...",CPD-3569,1.0,7020885,C(CC(=O)[O-])C(C(=O)[O-])NC(=O)C[NH3+],7,11,5,...,0.624729,1.0,0,0,1,0,0,0,0,0.748175
943,3.4.13.18,<rdkit.Chem.rdchem.Mol object at 0x000001FF0EC...,"[0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, ...",CPD0-1445,1.0,7006474,CC(C(=O)NC(CCC(=O)[O-])C(=O)[O-])[NH3+],8,13,5,...,0.624729,1.0,0,0,1,0,0,0,0,0.711864
617,1.2.1.38,<rdkit.Chem.rdchem.Mol object at 0x000001FF0EC...,"[0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, ...",CPD-469,1.0,6857435,CC(=O)NC(CCC=O)C(=O)[O-],7,10,4,...,0.193548,1.0,1,0,0,0,0,0,0,0.706564


from https://github.com/brilee/python_uct

In [5]:
class UCTNode(): #the node object, it corresponds to a molecular state.
    def __init__(self, mol_state, move, parent=None): #a given node needs a molecular state
        self.mol_state = mol_state #associated State() object which is a molecular structure
        self.move = move # if a node was made with a move, that move index is linked here. Move is a master index of sorts
        self.is_expanded = False # nodes start unexpanded
        self.parent = parent  # Optional[UCTNode] (if it's not the root it should have a parent)
        self.children = {}  # Dict[move, UCTNode] starts with no children
        
        self.precursors = np.empty([362], dtype=('U512'))#zeros([362], dtype=np.float32)
        self.enzymes = np.empty([362], dtype=('U512'))#zeros([362], dtype=np.float32)
        
        self.child_probabilities = np.zeros([362], dtype=np.float32) # an array of probabilities, indicating the current preference for each followup move
        self.child_number_visits = np.zeros([362], dtype=np.float32) # an array of the number of visits of each child
        self.children_total_values = np.zeros([362], dtype=np.float32) # an array of the total values of each of the children
        
    """Now that each node no longer knows about its own statistics, 
    we create aliases for a node’s statistics by using property getters and setters. 
    These allow us to transparently proxy these properties to the relevant entry in the parents’ child arrays."""
        
    @property
    def number_visits(self):
        return self.parent.child_number_visits[self.move]

    @number_visits.setter
    def number_visits(self, value):
        self.parent.child_number_visits[self.move] = value

    @property
    def total_value(self):
        return self.parent.children_total_values[self.move]

    @total_value.setter # self.move indexes the child to itself in the parent's children_total_values array. 
    def total_value(self, value):
        self.parent.children_total_values[self.move] = value

    def child_Q(self): # calculate Quality for the child arrays
        return self.children_total_values / (1 + self.child_number_visits)

    def child_U(self): # calculate Upper confidence bound for the child arrays
        return math.sqrt(self.number_visits) * (self.child_probabilities / (1 + self.child_number_visits))

    def best_child(self): #quickly finds the index of the child that has the highest aggregate of Q and U scores
        return np.argmax(self.child_Q() + self.child_U()) 

    def select_leaf(self):
        current = self
        while current.is_expanded: # if not expanded immediately return self
            best_move = current.best_child() # get the index of the best child
            current = current.maybe_add_child(best_move) # adds a child at the index if it isn't there already
        return current

    def expand(self, child_probabilities): # takes enzymes, precursors == (child structures), and probabilities. (all share index)
        self.is_expanded = True
        self.child_probabilities = child_probabilities

    def maybe_add_child(self, move):
        if move not in self.children:
            self.children[move] = UCTNode(
                #self.mol_state.react(move), move, parent=self) # make a child with a reaction, need to update react method to include structure and enzyme list or even combine the two
                self.mol_state.react(move, self.precursors, self.enzymes), move, parent=self)
        return self.children[move]

    def backup(self, value_estimate: float): #NEED to check 
        current = self
        while current.parent is not None:
            current.number_visits += 1
            current.total_value += value_estimate# used to be (value_estimate * self.mol_state.to_play)
            current = current.parent #MIGHT not be right

terminal_states = []#= ['C(=CC(=O)[O-])C(=O)[O-]', 'C(C(=O)C(=O)O)C(=O)O', 'C(CC(=O)O)C(=O)C(=O)O', 'C(C(=O)C(=O)O)C(=O)O',
                   #'C(C(=CC(=O)O)C(=O)O)C(=O)O', 'C(C(=O)O)C(CC(=O)O)(C(=O)O)O', 'C(C(C(C(=O)[O-])O)C(=O)[O-])C(=O)[O-]',
                   #'CC(C)(COP(=O)(O)OP(=O)(O)OCC1C(C(C(O1)N2C=NC3=C2N=CN=C3N)O)OP(=O)(O)O)C(C(=O)NCCC(=O)NCCSC(=O)CCC(=O)O)O']
    
for i in range(3):
    terminal_states.append(random.choice(s)) # currently semi-random SMILES

print('terminal states enumerated')            
            
def UCT_search(mol_state, num_reads):
    root = UCTNode(mol_state, move=None, parent=DummyNode()) # initializes the search starting with the given mol_state
# the parent is a dummy node because there shouldn't be a parent for the root molecule
    
    pathway_choice = []
    
    for _ in range(num_reads): # iterator is _ because it is not used in the loop
        leaf = root.select_leaf() # if this is the first visit (current is not expanded), the current is evaluated, else the best child is
        
        leaf.precursors, leaf.enzymes, child_probabilities, value_estimate = Metrics.evaluate(leaf.mol_state) # needs to include a rollout, Segler et al. handled rewards like such
        
        leaf.expand(child_probabilities)
        leaf.backup(value_estimate)
        
    print('tree search complete')    
    
    while root.mol_state.smiles not in terminal_states: # if the root is a terminal SMILES stop immediately
        # this method has no failsafe as it assumes there always is a path
        
        idx = np.argmax(root.child_number_visits) # find the index of the most visted child node
        print(idx)
        pathway_choice.append(root.children[idx]) # add that node to the pathway
        root = root.children[idx] # make the child node the root
        
    return pathway_choice
    
    #return np.argmax(root.child_number_visits) # returns the index of the root's child, which is the first retrosynthetic node

class DummyNode(object): # makes a node without any child values or parent
    def __init__(self): 
        self.parent = None
        self.children_total_values = collections.defaultdict(float) 
        self.child_number_visits = collections.defaultdict(float)

class Metrics(): #returns an array of probabilities for followup moves and a value estimation of the current node
    @classmethod
    def evaluate(self, mol_state):
        
        smiles = mol_state.smiles
        
        #df = dist_for_expansion(prom_df, smiles, 3)
        
        #precursors = df['product']#s #NEED to be filled from ML method
        precursors = s
        #enzymes = df['Enzyme'] #e #NEED to be filled from ML method
        enzymes = e
                
        if smiles in terminal_states:
            reward = 2 
        else: #perform rollout here
            #depth = 0
            #while depth not > k:
                #depth +=1
                #if smiles in terminal_states: # if the initial state is solved
                    #reward =1
                    #break
                #elif unsolvable and depth =1: #smiles null? # if the first layer is unsolvable kill the rollout
                    #reward =-1
                    #break
                #else: 
                    #smiles = randomly? or deterministically sample from child structures
                    #reward = 1/(1/depth+1) # approaches 1 as depth increases
                
            reward = np.random.random() #take out eventually
        
        return precursors, enzymes, np.random.random([362]), reward       

class State(): # corresponds to a molecular structure. contains a smiles string and rdkit mol
    # NEED to make sure move indexes map to reaction indexes 
    def __init__(self, smiles=None, enzyme=None): #smiles should be a smiles string, enzyme should be an EC#
        self.smiles = smiles #node.precursor[move] needs to point here
        self.mol = Chem.rdmolfiles.MolFromSmiles(smiles)
        self.enzyme = enzyme #node.mol_state.enzyme[move] needs to point here

    def react(self, move, precursors=s, enzymes=e): #change s and e
        return State(precursors[move], enzymes[move])# eventually build and return a new structure or point to a new smiles

terminal states enumerated


Testing: if the search results in an index error of 0 it probably means it reached a depth that hadn't properly been expanded? All arrays are intialized with zeroes

future needs to get rid of state class entirely and merge it into the node class

In [6]:
smiles = 'CC(=O)NC(CCC(=O)OP(=O)(O)O)C(=O)O' #'''CC1CCC2CC(C(=CC=CC=CC(CC(C(=O)C(C(C(=CC(C(=O)CC(OC(=O)C3CCCCN3C(=O)C(=O)C1(O2)O)C(C)CC4CCC(C(C4)OC)O)C)C)O)OC)C)C)C)OC'
state = State(smiles)
num_reads = 100
import time
tick = time.time()
node_list = UCT_search(state, num_reads)
tock = time.time()
print("Took %s sec to run %s times" % (tock - tick, num_reads))

print("parsing nodelist")

for member in node_list:
    print(member.mol_state.smiles)
    
print("parsing enzymes")

for member in node_list:
    print(member.mol_state.enzyme)

tree search complete
22
Took 0.025920867919921875 sec to run 100 times
parsing nodelist
c1cccc(PONC)c1
parsing enzymes
B7BV30PT
